In [1]:
import numpy as np
def apk(actual, predicted, k):
    if len(predicted)>k:
        predicted = predicted[:k]
    score = 0.0
    num_hits = 0.0
    for i,p in enumerate(predicted):
        if p in actual and p not in predicted[:i]:
            num_hits += 1.0
            score += num_hits / (i+1.0)
    if not actual:
        return 0.0
    return score / min(len(actual), k)


def mapk(actual, predicted, k):    
  return np.mean([apk(a,p,k) for a,p in zip(actual, predicted)])

actual = [['prod_1','prod_2'], ['prod_3','prod_4','prod_5']]
# Esta es vuestra predicción para ambos los clientes (ordenando los productos según vuestra propensión, de mayor a menor):
predicted = [['prod_2','prod_1','prod_6'], ['prod_5','prod_4','prod_3','prod_1','prod_6']]
# El resultado es perfecto, ya que en el top 2 y top 3 de nuestra predicción (para cada cliente) figuran los 2 y 3 productos que realmente compró cada cliente:
print(mapk(actual, predicted, 4))

1.0


In [5]:
inds=[f'ind_{i}' for i in range(1,25+1)]
predictions=[[8.2556161e-14, 4.8782150e-12, 7.6876330e-01, 1.9379141e-10,
        2.8975064e-02, 5.8960216e-03, 1.7235639e-03, 3.0751782e-03,
        8.7348372e-04, 6.5899411e-07, 3.6027931e-08, 3.1187183e-03,
        2.2808554e-02, 7.2535232e-04, 3.6250373e-07, 6.1518585e-05,
        2.1875539e-06, 9.8511623e-03, 5.2901641e-03, 1.3931792e-03,
        2.9343987e-06, 1.9032551e-02, 2.0134714e-02, 5.7409979e-02,
        6.9743921e-03]]
y_true=[[0., 0., 1., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       1., 0., 0., 0., 0., 0., 1., 0.]]

In [21]:
def preparacion(predictions,inds,y_trues):
    for p,t in zip(predictions,y_trues):
        assert len(inds) == len(p), "Las listas inds y predictions deben tener la misma longitud"
        assert len(inds) == len(t)

        # Crear una lista de tuplas (prediccion, etiqueta)
        predicciones_con_etiquetas = list(zip(p, inds))
        threshold=.01
        predicciones_filtradas = [tupla for tupla in predicciones_con_etiquetas if tupla[0] > threshold]
        # Ordenar las tuplas por el valor de la prediccion en orden descendente
        predicciones_ordenadas = [et for punt,et in sorted(predicciones_filtradas, key=lambda x: x[0], reverse=True)]

        t_p=[i for y_t,i in zip(t,inds) if y_t==1]

        yield (predicciones_ordenadas,t_p)

In [22]:
list(preparacion(predictions,inds,y_true))

[[['ind_3', 'ind_24', 'ind_5', 'ind_13', 'ind_23', 'ind_22'],
  ['ind_3', 'ind_18', 'ind_24']]]

In [133]:
import pandas as pd
data=pd.read_csv('../../Data/Limpio/dataset_limpio_parte1.csv')
data1=pd.read_csv('../../Data/Limpio/dataset_limpio_parte2.csv')
data2=pd.read_csv('../../Data/Limpio/dataset_limpio_parte3.csv')
data=pd.concat([data,data1,data2]) # Ajusta el nombre del archivo según corresponda
data.head()

/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_17595/932331467.py:2: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data=pd.read_csv('../../Data/Limpio/dataset_limpio_parte1.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_17595/932331467.py:3: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data1=pd.read_csv('../../Data/Limpio/dataset_limpio_parte2.csv')
/var/folders/4f/gbmqzwn51yg8gh7yyxzgpcl80000gn/T/ipykernel_17595/932331467.py:4: DtypeWarning: Columns (4,8) have mixed types. Specify dtype option on import or set low_memory=False.
  data2=pd.read_csv('../../Data/Limpio/dataset_limpio_parte3.csv')


,cod_persona,mes,pais,sexo,edad,fecha1,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,...,ind_prod16,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25
0,5514,2016-04-28,ES,H,44,2016-04-29,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
1,5541,2016-04-28,ES,H,60,2016-04-29,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
2,5631,2016-04-28,ES,V,53,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
3,5656,2016-04-28,ES,H,47,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
4,5738,2016-04-28,ES,V,48,2016-04-28,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0


In [129]:
from surprise import Dataset, Reader, KNNWithMeans
from surprise.model_selection import train_test_split
from surprise.accuracy import rmse

# Definimos el lector con el rango de valoraciones (asumiendo que es de 0 a 1)
reader = Reader(rating_scale=(0, 1))

# Cargamos el conjunto de datos desde el DataFrame usando el lector definido
data = Dataset.load_from_df(data[['cod_persona', 'fecha1', 'ind_prod1']], reader)

# Dividimos el conjunto de datos en entrenamiento y prueba (80% entrenamiento, 20% prueba)
trainset, testset = train_test_split(data, test_size=0.2, random_state=42)

# Utilizamos el algoritmo de filtrado colaborativo basado en secuencia temporal
algo = KNNWithMeans(k=40, sim_options={'name': 'pearson', 'user_based': True, 'min_support': 3})

# Entrenamos el modelo con el conjunto de entrenamiento
algo.fit(trainset)

# Realizamos predicciones en el conjunto de prueba
predictions = algo.test(testset)

# Evaluamos el rendimiento del modelo utilizando RMSE
accuracy = rmse(predictions)
print("RMSE:", accuracy)


Computing the pearson similarity matrix...
Done computing similarity matrix.
RMSE: 0.0000
RMSE: 2.1587142153188827e-06


[(153327, '2015-06-04', 0.0),
 (1300993, '2003-05-14', 0.0),
 (585034, '2011-10-25', 0.0),
 (56755, '2015-11-02', 0.0),
 (385814, '2013-08-20', 0.0),
 (298417, '2014-04-28', 0.0),
 (1107861, '2003-11-21', 0.0),
 (504959, '2012-08-09', 0.0),
 (1309036, '2001-05-09', 0.0),
 (673478, '2010-05-28', 0.0),
 (913724, '2006-09-11', 0.0),
 (265568, '2014-08-01', 0.0),
 (301756, '2014-04-03', 0.0),
 (274741, '2014-07-28', 0.0),
 (727083, '2009-01-16', 0.0),
 (1351489, '2000-10-31', 0.0),
 (971726, '2005-11-24', 0.0),
 (208395, '2014-10-30', 0.0),
 (1197604, '2002-04-29', 0.0),
 (304001, '2014-03-21', 0.0),
 (1398590, '2000-01-04', 0.0),
 (1444950, '1998-10-21', 0.0),
 (141445, '2015-07-22', 0.0),
 (1271425, '2001-10-15', 0.0),
 (593872, '2011-10-12', 0.0),
 (284467, '2014-07-14', 0.0),
 (265594, '2014-08-01', 0.0),
 (475988, '2012-10-09', 0.0),
 (370048, '2013-09-25', 0.0),
 (542135, '2012-04-09', 0.0),
 (244718, '2014-09-16', 0.0),
 (669745, '2010-06-23', 0.0),
 (710411, '2009-05-25', 0.0),
 (6

In [40]:
import pandas as pd
from mlxtend.frequent_patterns import apriori, association_rules


# Asegurarse de que las fechas estén en el formato adecuado
data['fecha1'] = pd.to_datetime(data['fecha1'])

# Ordenar los datos por usuario y fecha
data = data.sort_values(by=['cod_persona', 'fecha1'])

# Seleccionar solo las columnas relevantes para productos
productos_cols = [f'ind_prod{i}' for i in range(1, 26)]
transacciones = data.groupby('cod_persona')[productos_cols].all().astype(int)
frq_items = apriori(transacciones, min_support = 0.01, use_colnames = True) 

/Users/mariolamas/anaconda3/envs/entorno_prueba/lib/python3.11/site-packages/mlxtend/frequent_patterns/fpcommon.py:109: DeprecationWarning: DataFrames with non-bool types result in worse computationalperformance and their support might be discontinued in the future.Please use a DataFrame with bool type
  warnings.warn(


In [41]:
frq_items

,support,itemsets
0,0.555890,(ind_prod3)
1,0.049531,(ind_prod5)
2,0.107270,(ind_prod8)
3,0.034396,(ind_prod9)
4,0.022467,(ind_prod12)
5,0.058958,(ind_prod13)
6,0.012655,(ind_prod14)
7,0.040659,(ind_prod18)
8,0.019004,(ind_prod19)
9,0.019603,(ind_prod20)


In [119]:
rules = association_rules(frq_items, metric ="lift", min_threshold = .1) 
rules = rules.sort_values(['confidence', 'lift'], ascending =[False, False])
rules

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
51,(ind_prod22),(ind_prod23),0.021783,0.022147,0.021783,1.000000,45.153475,0.021301,inf,0.999628
83,"(ind_prod5, ind_prod22)",(ind_prod23),0.019731,0.022147,0.019731,1.000000,45.153475,0.019294,inf,0.997536
114,"(ind_prod24, ind_prod22)",(ind_prod23),0.016418,0.022147,0.016418,1.000000,45.153475,0.016054,inf,0.994175
120,"(ind_prod5, ind_prod24, ind_prod22)",(ind_prod23),0.015199,0.022147,0.015199,1.000000,45.153475,0.014863,inf,0.992945
47,(ind_prod19),(ind_prod25),0.019004,0.036512,0.019004,1.000000,27.388173,0.018310,inf,0.982153
...,...,...,...,...,...,...,...,...,...,...
2,(ind_prod3),(ind_prod9),0.555890,0.034396,0.016482,0.029649,0.862004,-0.002639,0.995108,-0.264960
8,(ind_prod3),(ind_prod18),0.555890,0.040659,0.015734,0.028303,0.696110,-0.006869,0.987284,-0.495710
5,(ind_prod3),(ind_prod12),0.555890,0.022467,0.014237,0.025611,1.139941,0.001748,1.003227,0.276422
11,(ind_prod3),(ind_prod20),0.555890,0.019603,0.012249,0.022035,1.124077,0.001352,1.002487,0.248545


In [120]:
list(data['cod_persona'].unique())

[5514,
 5541,
 5631,
 5656,
 5738,
 5841,
 5863,
 5880,
 5902,
 5926,
 5931,
 5936,
 5956,
 6032,
 6035,
 6051,
 6107,
 6179,
 6219,
 6221,
 6230,
 6250,
 6309,
 6317,
 6337,
 6342,
 6344,
 6366,
 6388,
 6392,
 6418,
 6479,
 6483,
 6512,
 6536,
 6676,
 6683,
 6708,
 6732,
 6779,
 6824,
 6855,
 6867,
 6877,
 6943,
 6947,
 7087,
 7107,
 7135,
 7150,
 7174,
 7201,
 7261,
 7262,
 7265,
 7301,
 7331,
 7341,
 7418,
 7510,
 7517,
 7521,
 7550,
 7563,
 7573,
 7606,
 7628,
 7634,
 7635,
 7652,
 7669,
 7722,
 7741,
 7761,
 7821,
 7929,
 7978,
 8038,
 8069,
 8108,
 8122,
 8126,
 8144,
 8145,
 8154,
 8184,
 8297,
 8302,
 8327,
 8333,
 8417,
 8424,
 8431,
 8442,
 8459,
 8474,
 8478,
 8510,
 8545,
 8549,
 8553,
 8558,
 8579,
 8584,
 8599,
 8610,
 8617,
 8621,
 8636,
 8732,
 8741,
 8750,
 8771,
 8815,
 8822,
 8825,
 8858,
 8894,
 8909,
 8940,
 8952,
 8975,
 8987,
 8993,
 9039,
 9059,
 9087,
 9108,
 9116,
 9145,
 9225,
 9227,
 9228,
 9260,
 9264,
 9315,
 9379,
 9392,
 9507,
 9511,
 9590,
 9622,
 9674,

In [121]:
data_user=data[data['cod_persona']==33542].sort_values('mes')
inds=[f'ind_{i}' for i in range(1,26)]
data_user

,cod_persona,mes,pais,sexo,edad,fecha1,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,...,ind_prod16,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25
2627,33542,2015-12-28,ES,H,39,2015-12-30,N,1.0,0,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
2628,33542,2016-01-28,ES,H,39,2015-12-30,N,1.0,1,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
2629,33542,2016-02-28,ES,H,39,2015-12-30,N,1.0,2,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
2630,33542,2016-03-28,ES,H,39,2015-12-30,N,1.0,3,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0
2631,33542,2016-04-28,ES,H,39,2015-12-30,N,1.0,4,1.0,...,0,0,0,0,0,0,0.0,0.0,0,0


In [122]:
import numpy as np
res=[inds[elem-1] for elem in list(np.where(data_user.iloc[3,-25:]==1)[0])]
reales=[inds[elem-1] for elem in list(np.where(data_user.iloc[3+1,-25:]==1)[0])]
res,reales

([], [])

In [123]:
# Lista de productos que ha comprado el usuario
productos_comprados = res

# Filtrar las reglas de asociación
reglas_filtradas = rules[rules['antecedents'].apply(lambda x: all(producto in x for producto in productos_comprados))]

reglas_filtradas

,antecedents,consequents,antecedent support,consequent support,support,confidence,lift,leverage,conviction,zhangs_metric
51,(ind_prod22),(ind_prod23),0.021783,0.022147,0.021783,1.000000,45.153475,0.021301,inf,0.999628
83,"(ind_prod5, ind_prod22)",(ind_prod23),0.019731,0.022147,0.019731,1.000000,45.153475,0.019294,inf,0.997536
114,"(ind_prod24, ind_prod22)",(ind_prod23),0.016418,0.022147,0.016418,1.000000,45.153475,0.016054,inf,0.994175
120,"(ind_prod5, ind_prod24, ind_prod22)",(ind_prod23),0.015199,0.022147,0.015199,1.000000,45.153475,0.014863,inf,0.992945
47,(ind_prod19),(ind_prod25),0.019004,0.036512,0.019004,1.000000,27.388173,0.018310,inf,0.982153
...,...,...,...,...,...,...,...,...,...,...
2,(ind_prod3),(ind_prod9),0.555890,0.034396,0.016482,0.029649,0.862004,-0.002639,0.995108,-0.264960
8,(ind_prod3),(ind_prod18),0.555890,0.040659,0.015734,0.028303,0.696110,-0.006869,0.987284,-0.495710
5,(ind_prod3),(ind_prod12),0.555890,0.022467,0.014237,0.025611,1.139941,0.001748,1.003227,0.276422
11,(ind_prod3),(ind_prod20),0.555890,0.019603,0.012249,0.022035,1.124077,0.001352,1.002487,0.248545


In [124]:
# Ordenar las reglas filtradas por confianza o lift
regla_mejor = reglas_filtradas.sort_values(by=['confidence'], ascending=False).iloc[0]

# Seleccionar el producto consecuente como recomendación
recomendacion = regla_mejor['consequents']

print("La mejor recomendación para el usuario basada en sus compras previas es:")
print(recomendacion)
print(reales)

La mejor recomendación para el usuario basada en sus compras previas es:
frozenset({'ind_prod23'})
[]


## **Prueba simple**

In [136]:
inds=[f'ind_prod{i}' for i in range(1,26)]
data['productos'] = data.apply(lambda x: [ind for ind in inds if x[ind] > 0], axis=1)

data = {'user_id': [1, 1, 2, 2, 2, 3, 3, 3, 3, 4, 4],
        'product_id': [['A', 'B'], ['B', 'C'], ['B', 'C'], ['C', 'D'], ['D', 'E'],
                       ['A', 'B'], ['C', 'D'], ['D', 'E'], ['E', 'F'], ['A', 'C'], ['E', 'F']]}

In [145]:
import pandas as pd
import numpy as np
from sklearn.metrics.pairwise import cosine_similarity
from scipy import sparse

# Load the purchase history data into a DataFrame

purchase_history = pd.DataFrame(data)

# Count the number of purchases for each user and product combination
purchase_counts = purchase_history.explode('productos').groupby(['cod_persona', 'productos']).size().unstack(fill_value=0)

# Convert the purchase counts to a sparse matrix
sparse_purchase_counts = sparse.csr_matrix(purchase_counts)

# Compute the cosine similarity matrix between the products
cosine_similarities = cosine_similarity(sparse_purchase_counts.T)

# Define a function to recommend items for a user based on their purchase history
def recommend_items(user_id, n=5):
    # Get the user's purchase history
    user_history = sparse_purchase_counts[user_id].toarray().flatten()

    # Compute the cosine similarity between the user's purchased items and all other items
    similarities = cosine_similarities.dot(user_history)

    # Get the indices of the user's purchased items
    purchased_indices = np.where(user_history > 0)[0]

    # Set the similarity scores for purchased items to 0
    similarities[purchased_indices] = 0

    # Sort the items by similarity score and return the top n items
    recommended_indices = np.argsort(similarities)[::-1][:n]
    recommended_items = list(purchase_counts.columns[recommended_indices])
    
    # Remove the items that the user has already purchased
    purchased_items = list(purchase_counts.columns[purchase_counts.loc[user_id] > 0])
    recommended_items = [item for item in recommended_items if item not in purchased_items]

    # Return also the similarity scores for recommended items
    similarity_scores = similarities[recommended_indices]

    return recommended_items, similarity_scores





In [146]:
recomendacion(1)

TypeError: 'frozenset' object is not callable

In [140]:
data[data['cod_persona']==33841]

,cod_persona,mes,pais,sexo,edad,fecha1,xti_empleado,xti_nuevo_cliente,num_antiguedad,xti_rel,...,ind_prod17,ind_prod18,ind_prod19,ind_prod20,ind_prod21,ind_prod22,ind_prod23,ind_prod24,ind_prod25,productos
2692,33841,2015-12-28,ES,V,44,2015-12-29,N,1.0,0,1.0,...,0,0,0,0,0,0.0,0.0,0,0,[]
2693,33841,2016-01-28,ES,V,44,2015-12-29,N,1.0,1,1.0,...,0,0,0,0,0,0.0,0.0,0,0,[]
2694,33841,2016-02-28,ES,V,44,2015-12-29,N,1.0,2,1.0,...,0,0,0,0,0,0.0,0.0,0,0,[]
2695,33841,2016-03-28,ES,V,44,2015-12-29,N,1.0,3,1.0,...,0,0,0,0,0,0.0,0.0,0,0,[]
2696,33841,2016-04-28,ES,V,44,2015-12-29,N,1.0,4,1.0,...,0,0,0,0,0,0.0,0.0,0,0,[]
